In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog
import pandas as pd

def load_data():
    global dynamic_columns
    file_path = filedialog.askopenfilename(
        initialdir='/home/me/Downloads',  # Set the initial directory to your Downloads folder
        title="Select a CSV file", 
        filetypes=(("CSV files", "*.CSV"), ("All files", "*.*"))  # Include both lowercase and uppercase extensions
    )

    if not file_path:  # If no file is selected, exit the function
        return
    
    df = pd.read_csv(file_path)
    
    # Dynamically adjust column names based on the DataFrame
    dynamic_columns = df.columns.tolist()
    # Ensure the Treeview has enough columns for the DataFrame
    tree["columns"] = dynamic_columns
    
    # Define headings dynamically and ensure the 'Amount' column is identified
    for col in dynamic_columns:
        tree.heading(col, text=col)
        tree.column(col, anchor='w')
    
    # Insert data into the Treeview
    for index, row in df.iterrows():
        tree.insert('', 'end', values=[row[col] for col in dynamic_columns])

def update_total():
    total = 0
    # First, clear all existing "highlighted" tags
    for child in tree.get_children():
        print(tree.item(child,'tags'))
        current_tags = list(tree.item(child, 'tags'))
        if 'highlighted' in current_tags:
            current_tags.remove('highlighted')
        tree.item(child, tags=current_tags)

    try:
        # Access the 'Amount' column dynamically based on its position
        amount_col_index = dynamic_columns.index("Amount")
    except ValueError:
        print("The 'Amount' column is not found. Available columns are:", dynamic_columns)
        return  # Exit the function as we cannot proceed without knowing the correct column

    for child in tree.get_children():
        if 'checked' in tree.item(child)['tags']:
            total += float(tree.item(child)['values'][amount_col_index])
            # Apply the "highlighted" tag to this row, ensure 'checked' tag remains
            current_tags = list(tree.item(child, 'tags'))
            if 'highlighted' not in current_tags:
                current_tags.append('highlighted')
            tree.item(child, tags=current_tags)
    total_label.config(text=f"Total: {total:.2f}")


def check_row(event):
    row_id = tree.identify_row(event.y)
    if row_id:  # Ensure that a row was actually clicked
        if 'checked' in tree.item(row_id)['tags']:
            tree.item(row_id, tags=['unchecked'])
        else:
            tree.item(row_id, tags=['checked'])
        
        # Update total after changing the checked state
        update_total()

        # Deselect the row to remove the blue selection highlight
        tree.selection_remove(row_id)


# Create the main window
root = tk.Tk()
root.title("Bank Statement")

# Create a frame for the Treeview
frame = tk.Frame(root)
frame.pack(fill='both', expand=True)

# Initialize dynamic_columns to store column names after file is loaded
dynamic_columns = []

# Create the Treeview
tree = ttk.Treeview(frame, show='headings')
tree.pack(side='left', fill='both', expand=True)

tree.tag_configure('highlighted', background='yellow')


# Scrollbar for the Treeview
scrollbar = ttk.Scrollbar(frame, orient='vertical', command=tree.yview)
scrollbar.pack(side='right', fill='y')
tree.configure(yscrollcommand=scrollbar.set)

# Total label
total_label = tk.Label(root, text="Total: 0.00")
total_label.pack()

# Add checkboxes (simulated by double-click) and a method to handle checking
tree.bind('<Double-1>', check_row)



# Load data after selecting file
root.after(100, load_data)

root.mainloop()
